In [1]:
import glob, os
import re

In [2]:
#move to subject 0 folder
os.chdir("0")

In [3]:
#get all filenames for signs number 0-10
#remove the [0-9] to view all
files = glob.glob("[0-9]*_*.csv")
files

['0_Left.csv',
 '0_Right.csv',
 '10_Left.csv',
 '10_Right.csv',
 '1_Left.csv',
 '1_Right.csv',
 '2_Left.csv',
 '2_Right.csv',
 '3_Left.csv',
 '3_Right.csv',
 '4_Left.csv',
 '4_Right.csv',
 '5_Left.csv',
 '5_Right.csv',
 '6_Left.csv',
 '6_Right.csv',
 '7_Left.csv',
 '7_Right.csv',
 '8_Left.csv',
 '8_Right.csv',
 '9_Left.csv',
 '9_Right.csv']

In [4]:
#make a list of tuples of hand pairings
lr_pairs = list(zip(*[iter(files)]*2))
lr_pairs

[('0_Left.csv', '0_Right.csv'),
 ('10_Left.csv', '10_Right.csv'),
 ('1_Left.csv', '1_Right.csv'),
 ('2_Left.csv', '2_Right.csv'),
 ('3_Left.csv', '3_Right.csv'),
 ('4_Left.csv', '4_Right.csv'),
 ('5_Left.csv', '5_Right.csv'),
 ('6_Left.csv', '6_Right.csv'),
 ('7_Left.csv', '7_Right.csv'),
 ('8_Left.csv', '8_Right.csv'),
 ('9_Left.csv', '9_Right.csv')]

In [5]:
import pandas as pd

df_list = []

for pair in lr_pairs:
    df_left = pd.read_csv(pair[0], index_col=None).drop(['Unnamed: 0'], axis = 1)
    df_right = pd.read_csv(pair[1], index_col=None).drop(['Unnamed: 0'], axis = 1)
    
    #rename columns
    df_left = df_left.add_prefix('left')
    df_right = df_right.add_prefix('right')
    
    #merge
    df = pd.merge(df_left, df_right, left_on='leftTime', right_on='rightTime').drop('rightTime', axis = 1)

    #covert fist column to time object
    df['leftTime'] = pd.to_datetime(df['leftTime'].str[:-3], format = '%H:%M:%S.%f')
    
    #difference between rows
    df = df.diff().iloc[1:]
    df['leftTime'] = df['leftTime'].dt.total_seconds()
    
    #add class sign using regex of file name
    df.insert(loc = 0, column = 'Sign', value = re.findall('^[^_]+(?=_)', pair[0])[0])

#this makes velocity columns

#     for col in df.iloc[:,2:]:
#         df[col + ' velocity'] = df[col]/df['Time']
    
    df_list.append(df)

In [6]:
len(df_list)

11

In [7]:
df_list[2]

,Sign,leftTime,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,left thumbEF_L_Y,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,1,0.035,-0.000300,0.000178,-0.000275,-0.000289,0.000205,-0.000265,-0.000297,0.000241,...,-0.000406,-0.000140,0.000135,-0.000367,-0.000394,-0.000452,-0.000092,-0.000430,-0.000526,-0.000054
2,1,0.035,-0.000016,-0.000125,-0.000183,-0.000022,-0.000123,-0.000184,-0.000033,-0.000101,...,-0.000375,-0.000214,0.000106,-0.000261,-0.000131,-0.000800,0.000154,-0.000186,-0.000858,0.000268
3,1,0.035,0.000275,-0.000298,-0.000171,0.000317,-0.000340,-0.000176,0.000340,-0.000337,...,-0.000398,-0.000338,0.000122,-0.000229,-0.000149,-0.000904,0.000244,-0.000241,-0.001017,0.000414
4,1,0.036,0.000100,-0.000188,-0.000244,0.000138,-0.000229,-0.000249,0.000157,-0.000216,...,-0.000302,-0.000061,0.000299,-0.000151,0.000744,0.000129,-0.000092,0.000693,0.000126,0.000059
5,1,0.023,0.000074,-0.000023,-0.000080,0.000099,-0.000038,-0.000080,0.000112,-0.000038,...,-0.000085,0.000006,0.000163,-0.000010,0.000672,0.000525,-0.000195,0.000651,0.000540,-0.000120
6,1,0.036,-0.000109,0.000479,-0.000017,-0.000103,0.000501,-0.000008,-0.000077,0.000469,...,-0.000096,-0.000253,-0.000019,0.000014,0.000879,0.000589,-0.000289,0.000862,0.000652,-0.000179
7,1,0.039,-0.000014,-0.000384,0.000155,-0.000027,-0.000401,0.000147,-0.000034,-0.000398,...,-0.000029,-0.000298,0.000075,0.000004,0.000729,0.000881,-0.000382,0.000769,0.001039,-0.000349
8,1,0.043,-0.000154,-0.000348,0.000160,-0.000130,-0.000366,0.000159,-0.000116,-0.000345,...,-0.000179,-0.000325,0.000243,-0.000200,0.000063,0.000199,-0.000183,0.000131,0.000391,-0.000204
9,1,0.039,-0.000030,-0.000087,-0.000049,-0.000038,-0.000066,-0.000044,-0.000029,-0.000051,...,-0.000327,-0.000680,0.000328,-0.000265,-0.000157,-0.000112,-0.000065,-0.000137,0.000017,-0.000003
10,1,0.039,0.000063,0.000170,-0.000194,0.000065,0.000211,-0.000179,0.000062,0.000242,...,-0.001224,-0.001284,0.000316,-0.000320,0.001309,0.001166,-0.000712,0.000597,-0.000013,0.000192


In [8]:
sign_frame = pd.concat(df_list, axis=0, ignore_index=False)

In [9]:
#note that this is still just for subject zero
#you need to loop to get all subjects
sign_frame

,Sign,leftTime,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,left thumbEF_L_Y,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,0,0.028,0.000388,0.000133,0.000050,0.000406,1.526500e-04,6.270000e-05,4.030000e-04,0.000154,...,-0.000321,-0.000166,0.000114,-0.000298,-8.360000e-05,-0.000393,-0.000142,-0.000071,-0.000331,-0.000119
2,0,0.036,0.000420,0.000077,-0.000248,0.000443,8.508000e-05,-2.392000e-04,4.293000e-04,0.000095,...,-0.000235,-0.000095,0.000121,-0.000193,4.000000e-05,-0.000276,-0.000075,0.000035,-0.000261,-0.000034
3,0,0.033,0.000223,0.000264,-0.000160,0.000164,2.754900e-04,-1.697000e-04,1.210000e-04,0.000262,...,-0.000251,-0.000084,0.000163,-0.000204,-1.250000e-04,-0.000481,0.000008,-0.000147,-0.000522,0.000054
4,0,0.034,0.000103,0.000078,-0.000318,0.000105,1.032500e-04,-3.074000e-04,5.420000e-05,0.000168,...,-0.000213,-0.000179,0.000077,-0.000056,1.000000e-07,-0.000630,0.000164,-0.000104,-0.000869,0.000321
5,0,0.036,0.000055,0.000233,-0.000030,0.000002,2.627400e-04,-3.110000e-05,-2.190000e-05,0.000241,...,-0.000001,0.000016,0.000356,0.000150,5.750000e-04,0.000152,0.000179,0.000504,0.000025,0.000330
6,0,0.034,0.000057,-0.000107,-0.000086,0.000036,-1.058300e-04,-9.020000e-05,4.800000e-06,-0.000082,...,0.000182,-0.000167,0.000266,0.000380,7.489000e-04,0.000707,0.000175,0.000641,0.000499,0.000373
7,0,0.043,-0.000299,0.000086,0.000141,-0.000317,1.049400e-04,1.441000e-04,-2.997000e-04,0.000072,...,0.000063,-0.000336,0.000066,0.000256,3.090000e-05,0.000101,0.000222,-0.000133,-0.000282,0.000415
8,0,0.035,-0.000311,0.000298,0.000167,-0.000237,2.744500e-04,1.756000e-04,-1.455000e-04,0.000191,...,0.000094,-0.000228,-0.000003,0.000177,2.029000e-04,0.000241,0.000071,0.000158,0.000160,0.000154
9,0,0.037,-0.000337,0.000358,0.000164,-0.000282,3.761800e-04,1.841000e-04,-1.761000e-04,0.000299,...,0.000162,-0.000677,-0.000103,0.000404,1.943000e-04,0.000342,0.000210,0.000015,-0.000028,0.000452
10,0,0.040,-0.000230,0.000202,0.000275,-0.000232,2.447200e-04,2.916000e-04,-1.618000e-04,0.000205,...,-0.000147,-0.002017,-0.000730,0.000684,5.690000e-04,0.000138,0.000279,-0.000138,-0.001246,0.001110


In [10]:
#this is the still hand for signing numbers
sign_frame.groupby('Sign')['left thumbProximal_L_X'].agg('mean')

Sign
0     9.093222e-06
1     5.417744e-06
10    1.764326e-05
2    -3.434540e-07
3    -1.805375e-05
4    -1.074257e-05
5     4.638676e-05
6     1.597123e-05
7    -2.376928e-05
8    -2.341019e-07
9     1.554359e-05
Name: left thumbProximal_L_X, dtype: float64

In [11]:
#moving right hand
sign_frame.groupby('Sign')['right thumbProximal_L_Y'].agg('mean')

Sign
0     0.000042
1     0.000038
10   -0.000151
2     0.000007
3     0.000023
4     0.000044
5     0.000062
6    -0.000139
7     0.000004
8    -0.000072
9    -0.000066
Name: right thumbProximal_L_Y, dtype: float64